<a href="https://colab.research.google.com/github/KojoBarbie/CNN_practice_1/blob/main/CNN_practice_img.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNNを用いて画像分類
pokeapiを使用して集めたポケモンの写真をもとに、そのポケモンが伝説のポケモンであるかどうかを判定する画像分類を作る。
このノートブックでは、画像の収集を行う。

## import
必要なモジュールなどのインポート

In [ ]:
# Googleドライブをマウント(Colabの場合)
from google.colab import drive
drive.mount('/content/drive')

# 定番
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pprint

# 画像収集の時に使う
import urllib.error
import urllib.request

# 水増しの時に使う
import cv2
import glob

Mounted at /content/drive


## 画像を収集する
pokeapiから画像のリンクを開き、ダウンロードする  
pokeapiのリンク: https://pokeapi.co/

In [ ]:
# 画像をURLからダウンロードする関数
# 参考: https://note.nkmk.me/python-download-web-images/
def download_file(url, file_name):
    try:
        with urllib.request.urlopen(url) as web_file, open(file_name, 'wb') as local_file:
            local_file.write(web_file.read())
    except urllib.error.URLError as e:
        print(e)

In [ ]:
# pokeapiを利用して画像を収集する
def collect_png(rootpath, rooturl):
  for i in range(1, 899):
    url = rooturl + str(i)+ ".png"
    path = rootpath + str(i).zfill(3) + ".png" 
    download_file(url, path)
    if i % 100 == 0:
      print(str(i) + "まで終わった")

In [ ]:
# ひとまずartworkだけダウンロードしてみる
rootpath = "/content/drive/MyDrive/data/artwork/"
rooturl = "https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/other/official-artwork/"

collect_png(rootpath, rooturl)

100まで終わった
200まで終わった
300まで終わった
400まで終わった
500まで終わった
600まで終わった
700まで終わった
800まで終わった


In [ ]:
# homeもダウンロード
rootpath = "/content/drive/MyDrive/data/home/"
rooturl = "https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/other/home/"

collect_png(rootpath, rooturl)

100まで終わった
200まで終わった
300まで終わった
400まで終わった
500まで終わった
600まで終わった
700まで終わった
HTTP Error 404: Not Found
800まで終わった


## 画像の水増し
約1800枚の画像を入手できたが、伝説のポケモン（UBなども入れて79匹）のデータが少し足りないので、水増しする。  
色調転換・左右反転・左右90度回転の4つを組み合わせて、1枚の写真から16枚の写真に水増しした。

In [ ]:
def show_img(img):
  show_img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) # 表示のためBGRをRGBに変換する。
  plt.imshow(show_img) # matplot.libを用いて読み込んだ画像を表示します。

In [ ]:
# 参考コードそのまま
def scratch_image(img, flip=True, cvt=True, cvt2=True, bit=True, warp=True):
    # 水増しの手法を配列にまとめる
    methods = [flip, cvt, bit, warp]

    # flip は画像の左右反転
    # cvt  は色調変換
    # cvt2 は色調変換
    # bit  は色反転
    # warp は90度回転

    images = [img]
    # 手法に用いる関数
    scratch = np.array([

        #画像の左右反転のlambda関数
        lambda x: cv2.flip(x,1),

        #色調変換のlambda関数
        lambda x: cv2.cvtColor(x, cv2.COLOR_BGR2RGB),

        #色反転のlambda関数
        lambda x: cv2.bitwise_not(x),

        #90度回転するlambda関数
        lambda x: cv2.warpAffine(x, cv2.getRotationMatrix2D
        (tuple(np.array([img.shape[1], img.shape[0]]) / 2), 90, 1.0), img.shape[::-1][1:3])

    ])


    # 関数と画像を引数に、加工した画像を元と合わせて水増しする関数
    doubling_images = lambda f, imag: (imag + [f(i) for i in imag])

    # doubling_imagesを用いてmethodsがTrueの関数で水増ししてください
    for func in scratch[methods]:
        images = doubling_images(func,images)

    return images

In [ ]:
def copy_img(dir, file_name, output_path):
  img_path = "/content/drive/MyDrive/data/" +  dir + "/" + file_name
  # 画像の読み込み
  img = cv2.imread(img_path)
  #画像のトリミング
  center = (img.shape[0]//2, img.shape[1]//2)
  img = img[center[0] - np.min(center) : center[0] + np.min(center), center[1] - np.min(center) : center[1] + np.min(center)]
  #画像のリサイズ
  img = cv2.resize(img, (200,200))        
  # 画像の水増し
  scratch_images = scratch_image(img)
  # 拡張子なしファイル名を取得
  name = os.path.splitext(os.path.basename(img_path))[0]

  # output
  num = 0
  for im in scratch_images:
    # まず保存先のディレクトリを指定、番号を付けて保存
    cv2.imwrite(output_path + "/" + dir + "_" + name + "_" + str(num).zfill(2) + ".png" ,im)
    num += 1

In [ ]:
def makedata():
  count = 0
  dir_list = ["artwork", "home"]
  for i in range(1, 301):
    for dir in dir_list:
      try:
        copy_img(dir, str(i).zfill(3) + ".png", "/content/drive/MyDrive/input")
      except:
        print("error at " + str(i))
    count += 1
    time.sleep(2)
    if count % 50 == 0:
      print("finish by " + str(count))
      time.sleep(10)

In [ ]:
makedata()

## CSVファイルの作成
水増しした写真をもとに、写真ファイルのパスと伝説のポケモンか判別するラベルをつけたデータフレームを作成し、それをcsvで保存する。

In [ ]:
# memo
legends = [144, 145, 146, 150, 151, # 第1世代
           243, 244, 245, 249, 250, 251, # 第2世代
           377, 378, 379, 380, 381, 382, 383, 384, 385, # 第3世代
           480, 481, 482, 483, 484, 485, 486, 487, 488, 494, # 第4世代
           638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 649, # 第5世代
           716, 717, 718, 719, 721, # 第6世代
           772, 773, 785, 786, 787, 788, 789, 790, 791, 792, # 第7世代（伝説）
           794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809, # 第7世代（UB, 幻のポケモン）
           888, 889, 894, 895, 896, 897, 898] # 第8世代

len(legends)

79

In [ ]:
# 空のdataframeを作成
df = pd.DataFrame(index = [], columns = ["id", "img", "legend"])

db_path = "/content/drive/MyDrive/input"
file_list = glob.glob(db_path + "/*.png")

for img_file in file_list:
  name = os.path.splitext(os.path.basename(img_file))[0]
  name = int(name.split("_")[1])
  is_legend = 0
  if name in legends:
    is_legend = 1
  record = pd.Series([name, img_file, is_legend], index=df.columns)
  df = df.append(record, ignore_index=True)

df.head()

,id,img,legend
0,669,/content/drive/MyDrive/10_study/CNN/input/home...,0
1,669,/content/drive/MyDrive/10_study/CNN/input/home...,0
2,669,/content/drive/MyDrive/10_study/CNN/input/home...,0
3,669,/content/drive/MyDrive/10_study/CNN/input/home...,0
4,669,/content/drive/MyDrive/10_study/CNN/input/home...,0


In [ ]:
# dfをcsvファイルに保存
df.to_csv("/content/drive/MyDrive/data.csv")

csvファイルとして保存することができました。  
今後の課題として、学習の際にpngファイルをTensor型に直すことを見越して、あらかじめそのあたりの処理までできていたらいいかなと思います。